## **Questão 1**

#### **Explicando alguns termos**


##### **1) Backtest**

Backtest é o processo de testar uma estrátegia de investimentos ou modelo de trading usando dados históricos. O objetivo de um backtest é
avaliar como a estratégia elaborada teria se comportado no passado para, a partir disso, inferir como pode ser o potencial futuro dessa
estratégia.

##### **2) Dividendos**

Dividendos são pagamentos distribuidos por uma certa empresa a seus acionistas como uma parcela do lucro da empresa. Eles são uma forma de retorno de investimento para os acionistas e podem ser pagos em dinheiro ou em ações adicionais.

##### **3) Split**

Um split, ou split de ações, é um evento corporativo onde uma certa empresa decide dividir suas ações existentes em um número maior de ações. Isso geralmente é feito para tornar o preço dessas ações mais acessível aos investidores, sem alterar o valor total da empresa.

##### **4) Reverse split**

Um reverse split é um evento corporativo em que uma empresa reduz o número de suas ações em circulação ao consolidar várias ações existentes em uma única ação nova. Isso acaba resultando em um aumento no preço das ações de tal empresa, sem alterar o valor total de mercado da mesma.

##### **5) Spin-off**

Um spin-off é um evento corporativo em que uma empresa cria uma nova empresa independente, separando uma parte de seus negócios. Em tal evento, os acionistas da empresa original geralmente recebem ações da nova empresa, proporcionais a sua participação na empresa original.

##### **6) Adjustment close**

O adjustment close é basicamente o preço de uma ação ajustado a ocorrência de eventos corporativos como dividendos, splits, reverse splits e spin-offs. Esse tipo de ajuste é feito para fornecer uma visão mais precisa do desempenho de uma ação ao longo do tempo, levando em conta as mudanças que afetam o preço.

#### **Resolução da questão**

##### **Instalação de bibliotecas**

In [ ]:
# Instala a bibliotecas que ser utilizada para a importação dos dados necessários para a resolução das questões abaixo.
%pip install yfinance

##### **Importação de bibliotecas**

In [ ]:
# Importa as bibliotecas que serão utilizadas na resolução das questões abaixo.
import yfinance as yf
# Importa a biblioteca que será utilizada para a manipulação de DataFrames.
import pandas as pd
# Importa a biblioteca que será utilizada em operações que envolvam aleatoriedade.
import random
# Importa a biblioteca que será utilizada para a manipulação de datas.
from datetime import datetime
# Importa as bibliotecas que serão utilizadas para a criação de gráficos.
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns


##### **Setup**

In [ ]:
# Cria uma variável de configuração que conterá dados úteis na resolução das questões abaixo.
setup = {
    "start_date": datetime(2018,1,1).date(),
    "end_date": datetime(2022,12,31).date(),
    "tickers": {
        "Vale":"VALE3.SA",
        "Banco do Brasil": "BBAS3.SA",
        "Bradesco":"BBDC3.SA",
        "Itau":"ITUB4.SA",
        "BTG Pactual":"BPAC11.SA",
        "Banco Pan":"BPAN4.SA",
        "Banco de Brasilia":"BSLI3.SA",
        "Banco BMG":"BMGB4.SA",
        "Santander":"SANB11.SA",
        "BR Partners":"BRBI11.SA"
    }
}

##### **Questão 1 - a)** 

*Obtenha os dados diários das ações da Vale (VALE3)*

In [ ]:
def get_ticker_data(ticker, start_date, end_date):
    '''
        Description: Essa função é responsável por obter alguns dados sobre as ações de uma determinada empresa em um certo período.
        
        Args:
            ticker (string): Identificador da empresa que se deseja obter os dados.
            start_date (string): Primeira data em que se deseja obter os dados sobre as ações da empresa.
            end_date (string): última data em que se deseja obter os dados sobre as ações da empesa.
        
        Return:
            data (pandas.DataFrame): DataFrame que contém alguns dados sobre as ações da empresa em questão.
    '''
    try:
        # Obtem o DataFrame que contém alguns dados sobre as ações da empresa solicitada.
        data = yf.download(ticker, start_date, end_date)
    except Exception as e:
        print(f"Erro ao baixar os dados do ticker {ticker}: {e}")
    
    # Retorna o DataFrame que contém os dados obtidos.
    return data

In [ ]:
# Salva em uma variável o DataFrame que contém os dados obtidos.
VALE_data = get_ticker_data(setup["tickers"]["Vale"], setup["start_date"], setup["end_date"])

In [ ]:
# Exibo os dados obtidos.
VALE_data

##### **Questão 1 - b)**
*Obtenha dados diários de um conjunto de 10 ações da B3 (inclua a VALE3 e escolha as outras 9); agrupe-as da forma que preferir. Depois, filtre para obter um dataframe apenas com o preço de fechamento ajustado*

In [ ]:
def get_tickers_adjusted_close_price(tickers_list, start_date, end_date):
    '''
        Description: ...
        
        Args:
            tickers_list (list): Lista contendo as strings que representam os tickers das empresas cujos dados das ações serão obtidos.
            start_date (datetime.date()): Data inicial do intervalo onde os dados serão extraidos.
            end_date (datetime.date()): Data final do intervalo onde os dados serão extraidos.
        
        Return:
            result (pd.DataFrame): DataFrame contendo os preços de fechamento ajustado durante o período compreendido entre "start_date"
            e "end_date" de todas as empresas cujo ticker esta em "tickers_list".       
            
        Obs: Pode ocorrer que alguma das empresas cujo ticker esta em "tickers_list" tenha sido listada na B3 após a data "start_date", caso isso
        ocorra, o preço de fechamento ajustado de tal empresa será setado como 0 em todas as datas a partir de "start_date" até a data em que 
        essa empresa foi listada na B3.
    '''
    
    # Por conta da variável "index_union" que será criada mais abaixo, trataremos o download dos dados do primeiro ticker de forma separada.
    # Obtem os dados do primeiro ticker.
    first_ticket = yf.download(tickers_list[0], start_date, end_date)
    # Cria uma variável que conterá a união (no sentido de conjuntos) de todos os índices de cada um dos tickers pertencentes a "ticker_list".
    # Ou seja, essa variável guardará os índices (que são as datas onde ocorreu negociação na B3) do ticker pertencente a "ticker_list" que foi
    # listado primeiro na B3. Faremos isso visando a padronização citada na observação da docstring dessa função.
    index_union = first_ticket.index
    # Cria uma lista para guardar as séries que representam o "Adj Close" de cada um dos tickers em "ticker_list".
    series_list = []
    # Adiciona a série "Adj Close" do primeiro ticker em "ticker_list" a lista "series_list".
    series_list.append(first_ticket['Adj Close'])
    
    # Percorre todos os tickers em ticker_list com exceção do primeiro.
    for ticker in tickers_list[1:]:
        try:
            # Faz o download dos dados ticker em questão no período compreendido entre "start_date" e "end_date".
            ticker_data = yf.download(ticker, start_date, end_date)
        except Exception as e:
            print(f"Erro ao baixar os dados do ticker {ticker}: {e}")
            continue
        # Faz a união dos índices do ticker em questão com todos os índices dos tickers que foram unidos até então.
        index_union = index_union.union(ticker_data.index)
        # Adiciona a série "Adj Close" do ticker em questão a lista "series_list".
        series_list.append(ticker_data['Adj Close'])
        
    # Reindexa todas as séries contidas em "series_list" levando em conta os índices em "index_union" e preenche eventuais valores faltantes
    # como zero.
    reindexed_serie_list = [serie.reindex(index_union, fill_value = 0) for serie in series_list]
    
    # Concatena todas as séries que foram reindexadas acima em um único DataFrame e usa o ticker de cada uma das séries como chave para
    # criar níveis superiores.
    result = pd.concat(reindexed_serie_list, axis=1, keys=tickers_list)
    
    # Retorna o DataFrame criado acima.
    return result

In [ ]:
# Cria uma lista que contém todos os tickers que estão na variável de configuração "setup".
tickers_list = list(setup["tickers"].values())
# Obtem alguns dados relacionados as ações de cada um dos tickers presentes na "tickers_list" criada acima.
tickers_data = get_tickers_adjusted_close_price(tickers_list, setup['start_date'], setup['end_date'])

In [ ]:
# Exibe os dados obtidos.
tickers_data

##### **Questão 1 - c)**

*Faça um único gráfico contendo a série temporal do preço de fechamento ajustado de cada uma das 10 ações que você escolheu no item anterior.*

In [ ]:
def plot_all_adjusted_close_prices(data):
    '''
        Description:
            Esta função plota ao longo do tempo os preços de fechamento ajustados das ações presentes no DataFrame "data".
        Args:
            data (pandas.DataFrame): DataFrame contendo os preços de fechamento ajustado das ações. Tal DataFrame deve ter datas como índices e
            deve ter cada uma de suas colunas representando os preços de fechamento ajustados de um certo ticket.
        Return:
            None: A função exibe o gráfico gerado, mas não retorna nenhum valor.
    '''    
    
    # Cria uma lista de timestamps que representará o eixo x do gráfico que será plotado.
    x_axis = data.index.tolist()
    # Cria a figura onde será plotado o gráfico.
    fig = go.Figure()

    # Adiciona cada série temporal ao gráfico.
    for ticker in data.columns:
        # Cria uma lista com os preços de fechamento ajustados das ações do ticker em questão.
        y_axis = data[ticker].values
        # Plota o gráfico (Data x Valor da Ação) do ticker em questão.
        fig.add_trace(go.Scatter(x=x_axis, y=y_axis, mode="lines", name=ticker))
    
    # Atualiza o layout para permitir destaque ao clicar na legenda.
    fig.update_layout(
        title='Série temporal do preço de fechamento ajustado de algumas ações',
        xaxis_title='Data',
        yaxis_title='Valor da Ação',
        legend_title='Ações',
        hovermode='x unified',
        clickmode='event+select'
    )

    # Callback para destacar a linha ao clicar na legenda.
    fig.update_traces(
        marker=dict(size=12),
        selector=dict(mode='markers')
    )

    # Exibe o gráfico criado.
    fig.show()

In [ ]:
# Plota os gráficos (Data x Valor da Ação) de cada uma das empresas cujo ticker está presente em "tickers_data".
plot_all_adjusted_close_prices(tickers_data)

##### **Questão 1 - d)**

*Escolha uma das 10 ações e faça um gráfico contendo a média móvel dos últimos 20 dias do preço de fechamento, mais as bandas de Bollinger (dois desvios para cima e dois desvios para baixo). O que a média móvel representa e porque usar as bandas de Bollinger?*

In [47]:
def get_serie_moving_average(serie, interval):
    '''
        Description:
            Essa função calcula as médias móveis dos dados de uma pd.Series "serie" em uma janela de tempo "interval".
        Args:
            serie (pd.Series): Série (coluna) contendo os dados que serão utilizados para o cálculo.
            interval (int): Janela de dias em que as médias móveis serão calculadas.
        Return:
            result (pd.Series): Série (coluna) contendo as médias móveis de "interval" dias dos dados da série fornecida.
    '''
    
    # Calcula as médias móveis dos dados da pd.Series "serie" na janela de tempo "interval" e guarda os resultados na variável "result".
    result = serie.rolling(window=interval).mean()
    
    # Lembre que, pela definição de média móvel, se tivermos um conjunto de x dados e quisermos calcular a média móvel desses dados em uma janela 
    # de tempo t, então teremos (x - t + 1) pontos da média móvel como resultado. Por conta disso, existarão algumas linhas da variável "result"
    # que serão nulas. 
    
    # Remove as linhas nulas da variável "result".
    result = result.dropna()
    
    return result

In [48]:
def get_serie_moving_std(serie, interval):
    '''
        Description:
            Essa função calcula os desvios padrões móveis dos dados de uma pd.Series "serie" em uma janela de tempo "interval".
        Args:
            serie (pd.Series): Série (coluna) contendo os dados que serão utilizados para o cálculo.
            interval (int): Janela de dias em que os desvios padrões móveis serão calculados.
        Return:
            result (pd.Series): Série (coluna) contendo os desvios padrões móveis de "interval" dias dos dados da série fornecida.
    '''
    
    # Calcula os desvios padrões móveis dos dados da pd.Series "serie" na janela de tempo "interval" e guarda os resultados na variável "result".
    result = serie.rolling(window=interval).std()
    
    # Lembre que, pela definição de desvio padrão móvel, se tivermos um conjunto de x dados e quisermos calcular os desvios padrões móveis desses
    # dados em uma janela de tempo t, então teremos (x - t + 1) pontos de desvio padrão móvel como resultado. Por conta disso, existarão algumas
    # linhas da variável "result" que serão nulas. 
    
    # Remove as linhas nulas da variável "result".
    result = result.dropna()
    
    return result

In [49]:
def plot_serie_moving_average_with_boillinger_bands(serie, interval, ticker=""):
    '''
        Description:
            Essa função plota as médias móveis dos dados da serie temporal "serie", juntamente das bandas de bollinger de tais médias moveis. 
            Para tal plot, os índices da serie temporal "serie" são usados como eixo x. Por conta disso, é esperado que tais índices sejam
            objetos do tipo datetime ou similares.
        Args:
            serie (pd.Series): Série (coluna) contendo os dados que serão utilizados para o plot.
            interval (int): Intervalo cujas estatísticas móveis serão calculadas.
            ticker (string) [opcional]: String contendo o nome do ticker cujos dados serão plotados no gráfico.
        Return:
            A função exibe o gráfico gerado, mas não retorna nenhum valor.
    '''
    
    # Calcula a média móvel de "interval" dias para a série temporal recebida como parâmetro.
    serie_moving_average = get_serie_moving_average(serie, interval)
    # Calcula o desvio padrão móvel de "interval" dias para a série temporal recebida como parâmetro.
    serie_moving_std = get_serie_moving_std(serie, interval)
    
    # Cria uma lista de timestamps que representará o eixo x do gráfico que será plotado.
    x_axis = serie.index.tolist()
    
    # Calcula as bandas superior e inferior de Bollinger.
    aux = 2*serie_moving_std.values
    upper_band = serie_moving_average.values + aux
    lower_band = serie_moving_average.values - aux
    
    # Cria a figura onde será plotado o gráfico.
    fig = go.Figure()

    # Adiciona a série temporal do preço no plot.
    fig.add_trace(go.Scatter(x=x_axis, y=serie.values, mode="lines", name=f"Preço de fechamento do ativo",line=dict(color="black",width=2)))
    # Adiciona a série temporal da média móvel no plot.
    fig.add_trace(go.Scatter(x=x_axis, y=serie_moving_average.values, mode="lines", name=f"Média móvel de {interval} dias",line=dict(color="red",width=2)))
    # Adiciona as séries temporais das bandas superior e inferior de Bollinger no plot.
    fig.add_trace(go.Scatter(x=x_axis, y=upper_band, mode="lines", name=f"Banda de Bollinger superior", line=dict(color="blue", width=2)))
    fig.add_trace(go.Scatter(x=x_axis, y=lower_band, mode="lines", name=f"Banda de Bollinger inferior", line=dict(color="blue", width=2)))
    
    # Adiciona legendas aos eixos do gráfico.
    fig.update_layout(
        title=f"Evolução temporal do preço do ativo {ticker}",
        xaxis_title='Data',
        yaxis_title='Valor',
    )

    # Exibe o gráfico criado.
    fig.show()

In [50]:
# Seleciona aleatoriamente dois dos tickers existentes na variável de configuração "setup" (O segundo ticker será utilizado futuramente).
ticker1, ticker2 = random.sample(tickers_list,2)

# Obtem os preços de fechamento ajustado do ticker_1.
ticker1_adj_close_price = get_ticker_data(ticker1, setup['start_date'], setup['end_date'])['Adj Close']

[*********************100%%**********************]  1 of 1 completed


In [51]:
plot_serie_moving_average_with_boillinger_bands(ticker1_adj_close_price, 20, ticker1)

**O que a média móvel representa e porque usar as bandas de Bollinger?**

*A média móvel é uma técnica estatística utilizada para suavizar dados ao calcular a média de um conjunto de valores dentro de um intervalo específico de tempo. Isso reduz o impacto de flutuações e ruídos, permitindo uma melhor visualização de tendências e padrões na série temporal. Além disso, a média móvel é frequentemente utilizada para identificar sinais de compra e venda em análise técnica.*

*As Bandas de Bollinger são um indicador técnico que mede a volatilidade de uma série temporal. Elas são compostas por uma média móvel e duas bandas situadas dois desvios padrão acima e abaixo dessa média móvel. A ideia é que, em uma distribuição normal, aproximadamente 95% dos dados estão dentro desse intervalo. Isso permite aos analistas identificar condições de sobrecompra ou sobrevenda, além de ajudar a entender a volatilidade do mercado.*

##### **Questão 1 - e)**
*Faça um único gráfico contendo o preço de fechamento da ação escolhida e seu Volume (negociado no dia)*

In [ ]:
# Usaremos aqui a variável ticker_1 definida na questão 1 - d).

# Obtem os dados do ticker em questão.
ticker1_data = get_ticker_data(ticker1,setup["start_date"],setup["end_date"])

In [ ]:
def plot_close_prices_with_volume(closing_prices_serie: pd.Series, volume_serie: pd.Series) -> None:
    '''
        Description:
            Esta função cria um gráfico que exibe a evolução temporal dos preços de fechamento de um ativo financeiro,
            junto com o volume de negociação correspondente para cada dia. Os preços de fechamento são plotados em uma linha
            no eixo y principal, enquanto o volume de negociação é exibido como um histograma no eixo y secundário.
            
            A função utiliza dois eixos y para permitir a visualização simultânea de dados com escalas diferentes:
            - O eixo y principal mostra os preços de fechamento.
            - O eixo y secundário, sobreposto ao eixo y principal, mostra o volume de negociação.

        Args:
            closing_prices_serie (pd.Series): Série temporal contendo os preços de fechamento do ativo.
            volume_serie (pd.Series): Série temporal contendo os volumes de negociação do ativo.

        Return:
            A função exibe o gráfico gerado, mas não retorna nenhum valor.
    '''
    
    # PS: O site https://coolors.co/ foi usado para gerar algumas cores que serão usadas nessa função.
    
    # Verifica o tipo das variáveis recebidas por parâmetro.
    if not isinstance(closing_prices_serie, pd.Series):
        raise TypeError("O parâmetro 'closing_prices_serie' deve ser um objeto do tipo pd.Series.")
    if not isinstance(volume_serie, pd.Series):
        raise TypeError("O parâmetro 'volume_serie' deve ser um objeto do tipo pd.Series.")
    
    # Cria a figure onde o gráfico será exibido.
    fig = go.Figure()
    
    # Adiciona ao plot o histograma que mostrará o volume de negociação em cada dia.
    fig.add_trace(go.Bar(
        # Cria o eixo x do histograma (que será o mesmo eixo x do gráfico de linha).
        x=volume_serie.index,
        # Cria o eixo y secundário (y2) onde será plotado os valores do histograma.
        y=volume_serie.values,
        # Cria o nome do eixo y secundário (y2).
        name="Volume de negociação",
        # Escolhe a cor das barras.
        marker=dict(color="black"),
        # Seta um eixo y específico (secundário) para o gráfico de barras (com intuito de não atrapalhar a escala de nenhum dos dois gráficos).
        yaxis="y2",
    ))
    
    # OBS: Com o intuito de que o gráfico da série temporal de preços sobreponha o gráfico de barras, construimos primeiro o histograma e o 
    # colocamos no eixo y2 (secundário) e, após isso, construimos a série temporal de preços e a colocamos no eixo y1 (principal).
    
    # Adiciona a série temporal de preços ao plot.
    fig.add_trace(go.Scatter(
        # Cria o eixo x gráfico de linha (que é o mesmo eixo x do histograma).
        x=closing_prices_serie.index,
        # Cria o eixo y principal (y1) onde serão plotados os valores do gráfico de linha.
        y=closing_prices_serie.values,
        # Seta o tipo do gráfico como sendo um gráfico de linha.
        mode="lines",
        # Cria o nome do eixo y principal (y1) do gráfico de linha.
        name="Preço",
        # Seta a cor do gráfico de linhas.
        line=dict(color="#B26E63"),
        # Seta um eixo y específico (principal) para o gráfico de linha (com intuito de não atrapalhar a escala de nenhum dos dois gráficos).
        yaxis="y1",
    ))
    
    # Seta algumas configurações para o plot que será gerado.
    fig.update_layout(
        # Seta o título do plot.
        title="Preços de Fechamento e Volume de Negociação",
        # Seta o título do eixo x do plot.
        xaxis_title="Data",
        # Seta o título do eixo y do plot.
        yaxis_title="Preço e Volume de Negociação",
        # Define algumas configurações para o eixo y principal (y1).
        yaxis=dict(
            # Seta o título do eixo y principal (y1).
            title="Preço",
            overlaying='y2', # Indica que o eixo y atual deve se sobrepor ao eixo y secundário (y2).
        ),
        # Define algumas configurações para o eixo y secundário (y2).
        yaxis2=dict(
            # Seta o título do eixo y secundário (y2).
            title="Volume de Negociação",
            # Define o lado em que o eixo y secundário (y2) aparecerá
            side='right',
            # Opta por não exibir o grid (linhas horizontais brancas do gráfico) para os valores de escala do eixo y secundário (y2).
            showgrid = False
        ),
        # Seta a cor de fundo da área de plotagem
        plot_bgcolor='#1A1B25', 
        # Seta a cor de fundo da figura
        paper_bgcolor='white', # Essa é a cor padrão, só escrevi esse parâmetro aqui para saber que ele existe.
    )
    
    # Exibe o gráfico
    fig.show()

In [ ]:
# Obtem a série temporal dos preços de fechamento do ticket em questão.
ticker1_closing_prices_serie = ticker1_data['Adj Close']
# Obtem a série temporal do volume diário de negociação do ticket em questão.
ticker1_volume_serie = ticker1_data['Volume']

# Plota o gráfico dos (Preços de Fechamento x Volume de Negociação) para o ticket em questão.
plot_close_prices_with_volume(ticker1_closing_prices_serie,ticker1_volume_serie)

##### **Questão 1 - f)**

*Faça um gráfico de dispersão (do preço de fechamento ajustado) entre o ativo escolhido anteriormente e outro de sua escolha. Qual sua interpretação?*

In [52]:
# Usaremos aqui as variáveis ticker1 e ticker2 definidas na questão 1 - d).

# Como os dados de ticker1 (ticker1_data) já foram obtidos anteriormente, não precisamos repetir tal processo.

# Obtem os dados do ticker2.
ticker2_data = get_ticker_data(ticker2,setup["start_date"],setup["end_date"])

# Salva em uma variável a série temporal dos preços de fechamento ajustados associados as ações da empresa representada por "ticker2".
ticker2_adj_close_price = ticker2_data['Adj Close']

# No intuito de facilitar a geração de legendas para os gráficos, atribuiremos os valores de "ticker1" e "ticker2" ao atributo name das séries
# temporais de fechamento de preço associados a cada uma dessas variáveis.
ticker1_adj_close_price.name = ticker1
ticker2_adj_close_price.name = ticker2

[*********************100%%**********************]  1 of 1 completed

In [36]:
def plot_comparative_dispersion_graphs(serie1: pd.Series, serie2: pd.Series) -> None:
    '''
        Description:
            Essa função é responsável por exibir em um mesmo plot dois gráficos de dispersão, sendo cada um desses gráficos associados a uma
            das séries temporais recebidas como parâmetro
        Args:
            serie1 (pd.Series): Série temporal contendo os valores que serão plotados em um dos gráficos de dispersão. 
            serie2 (pd.Series): Série temporal contendo os valores que serão plotados no outro gráfico de dispersão.
        Return:
            Essa função exibe os gráficos descritos, mas não retorna nada.
            
        Obs: Para que o plot faça sentido, é quase que necessário que ambas as séries temporais "serie1" e "serie2" tenham os mesmos índices.
    '''
    
    # Cria a figure onde o gráfico será plotado.
    fig = go.Figure()
    
    # Plota o primeiro gráfico de dispersão.
    fig.add_trace(
        # Seta algumas características no gráfico em questão.
        trace=(go.Scatter(
            # Seta os valores do eixo x do primeiro gráfico de dispersão.
            x = serie1.index,
            # Seta os valores do eixo y do primeiro gráfico de dispersão.
            y = serie1.values,
            # Atribui um nome ao primeiro gráfico de dispersão.
            name = serie1.name,
            # Seta o gráfico atual como um gráfico de dispersão.
            mode="markers", # Modo responsável por gerar o gráfico de dispersão.
            # Atribui uma cor aos pontos do gráfico de dispersão em questão.
            line=dict(
                color="black"
            )
        ))
    )
    
    # Plota o segundo gráfico de dispersão
    fig.add_trace(
        # Seta algumas características no gráfico em questão.
        trace=(go.Scatter(
            # Seta os valores do eixo x do segundo gráfico de dispersão.
            x = serie2.index,
            # Seta os valores do eixo y do segundo gráfico de dispersão.
            y = serie2.values,
            # Atribui um nome ao segundo gráfico de dispersão.
            name = serie2.name,
            # Seta o gráfico atual como um gráficode dispersão.
            mode="markers", # Modo responsável por gerar o gráfico de dispersão.
            # Atribui uma cor aos pontos do gráfico de dispersão em questão.
            line=dict(
                color="red"
            )
        ))
    )
    
    # Seta algumas configurações no plot.
    fig.update_layout(
        # Seta o título do plot.
        title=f"Comparativo entre os gráficos de dispersão de {serie1.name} e {serie2.name}",
        # Seta o título do eixo x do plot.
        xaxis_title="Data",
        # Seta o título do eixo y do plot.
        yaxis_title="Valor"
    )
    
    # Exibe o plot gerado.
    fig.show()

In [53]:
# Plota os gráficos de dispersão dos dois tickers ("ticker1" e "ticker2"). 
plot_comparative_dispersion_graphs(ticker1_adj_close_price,ticker2_adj_close_price)

**Interpretação do gráfico de dispersão acima**

*Creio que a ideia de plotar dois gráficos de dispersão em uma mesmo plot é observar se existe alguma correlação entre as ações que cada um
dos gráficos representa. Sendo assim, com base na observação do gráfico acima, é correto dizer que existe certa correlação entre ambas as ações, e existem hipóteses fortes para isso, como por exemplo, ambas são ações bancárias. Contudo, é de suma importação lembrar que eventuais correlações não implicam causalidade.*

##### **Desafio: Faça um gráfico candle stick da ação escolhida.**